# Numerai

Numerai is a hedge fund that crowdsources their market predictions.  They disseminate data that is anonymized so that the data scientists who are working on the forecasting are not even aware of the features and labels they are working with.  The prediction problem is reduced to a classification of predicting a gain or loss.

In this chapter we will fit a variety models to Numerai data.

## Reading-In Data

Let's begin by reading-in the data and organizing our features and labels.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_data = pd.read_csv('../data/numerai_training_data.csv')
df_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,target
0,0.499664,0.951271,0.127110,0.469706,0.188336,0.113830,0.917618,0.398412,0.418910,0.452983,...,0.137192,0.201437,0.507708,0.919475,0.978169,0.177080,0.101372,0.722138,0.832319,0
1,0.099515,0.682824,0.867939,0.943828,0.505526,0.886766,0.530862,0.531002,0.980002,0.941859,...,0.642640,0.533367,0.616879,0.697038,0.741461,0.086690,0.109533,0.324666,0.552276,1
2,0.671993,0.383901,0.533011,0.690863,0.176539,0.600196,0.381543,0.648849,0.831643,0.861746,...,0.520068,0.660924,0.538882,0.160117,0.765317,0.301772,0.352097,0.638205,0.383552,0
3,0.578177,0.872357,0.679625,0.108961,0.945910,0.571062,0.891958,0.916592,0.141508,0.258504,...,0.037959,0.604539,0.974103,0.187519,0.938254,0.560129,0.136483,0.284507,0.199446,1
4,0.474311,0.639613,0.563562,0.169508,0.456858,0.580710,0.969811,0.357417,0.157594,0.251147,...,0.038095,0.770200,0.697395,0.792327,0.711650,0.177080,0.247403,0.666598,0.755557,0


In [ ]:
df_X = df_data.drop(columns='target').copy()

In [ ]:
df_y = df_data['target'].copy()

## The Data is Clean

One nice thing about working with the Numerai data is that it is clean and normalized.  

1. All the features are in the range of $[0, 1]$.
1. All the features have a mean of 0.50 and a standard deviation of 0.28.
1. The occurrence of labels is even at 50% gains and 50% losses.

In [ ]:
df_X.mean()

feature1     0.511372
feature2     0.492770
feature3     0.492105
feature4     0.499420
feature5     0.502291
feature6     0.493039
feature7     0.480280
feature8     0.494526
feature9     0.492926
feature10    0.489265
feature11    0.495725
feature12    0.510969
feature13    0.489852
feature14    0.509350
feature15    0.487469
feature16    0.509012
feature17    0.488944
feature18    0.484929
feature19    0.491757
feature20    0.509223
feature21    0.498371
dtype: float64

In [ ]:
df_X.std()

feature1     0.282260
feature2     0.287446
feature3     0.282481
feature4     0.284493
feature5     0.289867
feature6     0.287061
feature7     0.287526
feature8     0.288087
feature9     0.293945
feature10    0.287046
feature11    0.290922
feature12    0.285451
feature13    0.291276
feature14    0.290140
feature15    0.286997
feature16    0.289279
feature17    0.284790
feature18    0.290445
feature19    0.283742
feature20    0.291001
feature21    0.289637
dtype: float64

Notice that a guess of increase for all assets would yield an accuracy of 50.5%.

In [ ]:
df_y.mean()

0.5051702657807309

## Logistic Regression

The first model that we will fit is a simple `LogisticRegression`.  We will use a 10-fold cross-validation accuracy as our goodness of fit metric.

In [ ]:
from sklearn.linear_model import LogisticRegression
mdl_logistic_regression = LogisticRegression(C=1.0, random_state=0)

In [ ]:
%%time
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mdl_logistic_regression, df_X, df_y, cv=10, verbose=0)

CPU times: user 21 s, sys: 17.9 s, total: 39 s
Wall time: 5.1 s


We get a mean accuracy of about 52% which is just slightly higher than guessing a gain for all assets.

In [ ]:
np.mean(scores)

0.5219476744186047

## Random Forest

Next let's fit a `RandomForestClassifier`.  We use a 5-fold cross-validation accuracy  rather than 10-fold because these models take time to run.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
mdl_random_forest = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=0)

In [ ]:
%%time
scores = cross_val_score(mdl_random_forest, df_X, df_y, cv=5, scoring='accuracy', verbose=0)
scores

CPU times: user 44.4 s, sys: 0 ns, total: 44.4 s
Wall time: 44.4 s


array([0.51858389, 0.51723422, 0.52450166, 0.51650748, 0.52460548])

Similar to logistic regression we get a mean score of around 52%

In [ ]:
np.mean(scores)

0.5202865448504983

## XGBoost

The next model that we will fit is a gradient boosted tree with the **xgboost** package.  We use 5-fold cross-validation to assess model performance.

In [ ]:
from xgboost import XGBClassifier
mdl_xgboost = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.01, eval_metric='logloss')

In [ ]:
scores = cross_val_score(mdl_xgboost, df_X, df_y, cv=5, scoring='accuracy', verbose=0)
scores

array([0.51292566, 0.51775332, 0.51977782, 0.51629983, 0.515625  ])

Our mean accuracy score is slightly lower at 51.5%.

In [ ]:
np.mean(scores)

0.5164763289036545

## Validation Set

In the subsequent sections we will try a variety of neural networks.  In order to check out-of-sample accuracy, let's first create a holdout set with `train_test_split()`.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.20, random_state=0)

## Initial Neural Network

We are now ready to fit our first neural network.

In [ ]:
import random
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

2023-09-02 13:39:44.717146: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 13:39:44.941096: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 13:39:44.943020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 13:39:45.900127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Let's set our random seeds to get reproducible results.

In [ ]:
def set_seeds(seed=100):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
set_seeds()

Next, let's build up our dense feed-forward neural network.  We use 3 hidden layer with 16, 8, and 4 units, respectively.

In [ ]:
model = Sequential()
model.add(Dense(units=16, input_dim=len(df_X.columns), activation='relu'))
model.add(Dense(units=8, input_dim=len(df_X.columns), activation='relu'))
model.add(Dense(units=4, input_dim=len(df_X.columns), activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

Now we can fit our model.

In [ ]:
%%time
h = model.fit(X_train, y_train, epochs=10, verbose=False);

CPU times: user 30.1 s, sys: 2.14 s, total: 32.2 s
Wall time: 21.2 s


As we can see, our neural network model performs about the same as the previous models.

In [ ]:
model.evaluate(X_test, y_test)

602/602 [==============================] - 1s 813us/step - loss: 0.6916 - accuracy: 0.5192


[0.6915713548660278, 0.5192068219184875]

## Dropout

In this section we implement dropout regularization with our neural network.  We use a slightly different architecture with 2 hidden units of 64 and 32 units, respectively.

In [ ]:
from keras.layers import Dropout

In [ ]:
set_seeds()

In [ ]:
model = Sequential()
model.add(Dense(units=64, input_dim=len(df_X.columns), activation='relu'))
model.add(Dropout(rate=0.1, seed=100))
model.add(Dense(units=32, input_dim=len(df_X.columns), activation='relu'))
model.add(Dropout(rate=0.1, seed=100))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
%%time
model.fit(X_train, y_train, epochs=10, verbose=False);

CPU times: user 32.3 s, sys: 2.01 s, total: 34.3 s
Wall time: 21.2 s


Dropout reglarization doesn't seem to have much of an effect on our model.

In [ ]:
model.evaluate(X_test, y_test)

602/602 [==============================] - 1s 898us/step - loss: 0.6916 - accuracy: 0.5192


[0.6916125416755676, 0.5192068219184875]

## Regularization

Finally we perform `l1` regularization on our neural network.

In [ ]:
from keras.regularizers import l1, l2

In [ ]:
set_seeds()

In [ ]:
model = Sequential()
model.add(Dense(units=64, input_dim=len(df_X.columns), activation='relu', activity_regularizer=l1(0.0005)))
model.add(Dense(units=32, input_dim=len(df_X.columns), activation='relu', activity_regularizer=l1(0.0005)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
%%time
model.fit(X_train, y_train, epochs=10, verbose=False);

CPU times: user 29.2 s, sys: 1.75 s, total: 30.9 s
Wall time: 20.1 s


Once again, `l1` regularization doesn't seem improve performance that much.

In [ ]:
model.evaluate(X_test, y_test)

602/602 [==============================] - 1s 873us/step - loss: 0.6925 - accuracy: 0.5205


[0.6924766302108765, 0.5205045938491821]

---   

**Discussion Question:** Based on our results so far, which model would you select?

---   

**Code Challenge:** Try some different neural architectures and see if you can improve model performance.

---